In [1]:
from glob import glob
import re
import pandas as pd
from tqdm.notebook import tqdm
from functions.data import preprocess

In [2]:
data_dir = "data"
save = True

# Count People

In [3]:
files = sorted(glob("data/downloads/*"))

# takes 1.5 min
# concat downloaded data
df_list = []
for file in tqdm(files):
    date = re.match("data/downloads/turnstile_(?P<date>\d*).csv", file)["date"]
    df = pd.read_csv(file)
    
    # adjust for daylight savings
    if date >= '181117' and date <= '190309':
        is_est = True
    df = preprocess(df, is_est)

    # reshape into timeseries
    timeseries_df = (
        df.groupby(["datetime", "station"])[["entries", "exits"]].first().unstack()
    ) 

    df_list.append(timeseries_df)
    
timeseries_df = pd.concat(df_list)

In [11]:
# count people who pass through the station within 4-hour time blocks
timeseries_diff_df = timeseries_df.shift(-1)-timeseries_df
count_df = timeseries_diff_df['entries']+timeseries_diff_df['exits']
count_df = count_df.fillna(0).reset_index()

In [14]:
if save:
    count_df.to_csv(f'{data_dir}/station-counts.csv', index=None)

# Check

In [15]:
df = pd.read_csv(f'{data_dir}/station-counts.csv')
df.head()

,datetime,1 AV,103 ST,103 ST-CORONA,104 ST,110 ST,111 ST,116 ST,116 ST-COLUMBIA,121 ST,...,WHITEHALL S-FRY,WHITLOCK AV,WILSON AV,WINTHROP ST,WOODHAVEN BLVD,WOODLAWN,WORLD TRADE CTR,WTC-CORTLANDT,YORK ST,ZEREGA AV
0,2018-03-24 00:00:00,407.0,0.0,291.0,23.0,145.0,36.0,0.0,0.0,24.0,...,0.0,60.0,189.0,0.0,45.0,96.0,26.0,0.0,0.0,0.0
1,2018-03-24 04:00:00,397.0,0.0,435.0,0.0,134.0,0.0,0.0,0.0,1.0,...,0.0,43.0,147.0,0.0,4.0,234.0,57.0,0.0,0.0,0.0
2,2018-03-24 08:00:00,1601.0,0.0,708.0,0.0,334.0,4.0,0.0,0.0,0.0,...,0.0,106.0,439.0,0.0,0.0,368.0,310.0,0.0,0.0,0.0
3,2018-03-24 12:00:00,1885.0,0.0,821.0,0.0,549.0,0.0,0.0,0.0,0.0,...,0.0,145.0,588.0,0.0,0.0,413.0,401.0,0.0,0.0,0.0
4,2018-03-24 16:00:00,2006.0,0.0,982.0,0.0,575.0,0.0,0.0,0.0,1.0,...,0.0,174.0,698.0,0.0,1.0,382.0,393.0,0.0,0.0,0.0
